In [531]:
import glob
import pandas as pd
import re
import locale
from time import strptime
%matplotlib inline
import matplotlib.pyplot as plt
from patsy import dmatrices

In [532]:
# Create container pickle
pickle_list = glob.glob('*.pickle')
pickle_list[:3]

['the-numbers/1970.pickle',
 'the-numbers/1971.pickle',
 'the-numbers/1972.pickle']

In [533]:
# assimilate pickles into dataframe, remove first column, convert unicode to string
the_list = []
for pickle in pickle_list:
    dataframe_temp = pd.read_pickle(pickle)
    the_list.append(dataframe_temp)

df_numbers = pd.concat(the_list)
df_numbers = df_numbers.drop(df_numbers.columns[[0]], axis=1)
df_numbers.columns = ['date', 'year', 'title', 'budget', 'bo', 'bo_infl', 'bo_int', 'bo_world']

def unicode_to_string(df):
    """
    Will not work if columns contain any numeric formats
    Be sure to run all if using ipython or jupyter notebooks because the function will return an error if the result is already stored in memory
    """
    df_cols = list(df.columns.values)
    for col in df_cols:
        df[col] = [unicode(x).encode("utf-8") for x in df[col]]
    return df
df_numbers = unicode_to_string(df_numbers)
print df_numbers.shape
df_numbers.head(3)

(1218, 8)


,date,year,title,budget,bo,bo_infl,bo_int,bo_world
0,"Dec 31, 1970",1970,The Ballad of Tam Lin,$0,$0,$0,$0,$0
0,"Dec 31, 1971",1971,Sie tötete in Ekstase,$0,$0,$0,$0,$0
0,"Dec 31, 1972",1972,Whoever Slew Auntie Roo?,$0,$0,$0,$0,$0


In [534]:
# some columns are misaligned: title and following columns begin at year (meaning no year for these cols)

# filter out misaligned entries
misalign_filter =  df_numbers['year'].str.isnumeric()
df_numbers = df_numbers[misalign_filter]

In [535]:
# drop columns
df_numbers = df_numbers.drop(df_numbers.columns[[5, 7]], axis=1)

print df_numbers.shape
df_numbers.head(3)

(1016, 6)


,date,year,title,budget,bo,bo_int
0,"Dec 31, 1970",1970,The Ballad of Tam Lin,$0,$0,$0
0,"Dec 31, 1971",1971,Sie tötete in Ekstase,$0,$0,$0
0,"Dec 31, 1972",1972,Whoever Slew Auntie Roo?,$0,$0,$0


In [536]:
def replace_char(df, col, old, new):
    """
    df is dataframe, old is old char(s), new is new char(s)
    entries must all be strings
    """
    df[col] = [x.replace(old, new) for x in df[col]]
    return df

# Replace $ signs
df_numbers = replace_char(df_numbers, 'budget', '$', '')
df_numbers = replace_char(df_numbers, 'bo', '$', '')
df_numbers = replace_char(df_numbers, 'bo_int', '$', '')

print df_numbers.shape
df_numbers.head(3)

(1016, 6)


,date,year,title,budget,bo,bo_int
0,"Dec 31, 1970",1970,The Ballad of Tam Lin,0,0,0
0,"Dec 31, 1971",1971,Sie tötete in Ekstase,0,0,0
0,"Dec 31, 1972",1972,Whoever Slew Auntie Roo?,0,0,0


In [537]:
# convert strings to values and for dollar amounts convert comma delimited strings to numbers
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

df_numbers['year'] = [int(x) for x in df_numbers['year']]
df_numbers['budget'] = [locale.atoi(x) for x in df_numbers['budget']]
df_numbers['bo'] = [locale.atoi(x) for x in df_numbers['bo']]
df_numbers['bo_int'] = [locale.atoi(x) for x in df_numbers['bo_int']]

df_numbers['budget'] = [float(x) for x in df_numbers['budget']]
df_numbers['bo'] = [float(x) for x in df_numbers['bo']]
df_numbers['bo_int'] = [float(x) for x in df_numbers['bo_int']]

# lowercase titles
df_numbers['title'] = map(str.lower, df_numbers['title'])

# calculate total box
df_numbers['total_bo'] = df_numbers['bo'] + df_numbers['bo_int']

# get month
df_numbers['month'] = df_numbers['date'].str[:3]
df_numbers['month'] = [strptime(x, '%b').tm_mon for x in df_numbers['month']]

print df_numbers.shape
df_numbers.head(3)

(1016, 8)


,date,year,title,budget,bo,bo_int,total_bo,month
0,"Dec 31, 1970",1970,the ballad of tam lin,0.0,0.0,0.0,0.0,12
0,"Dec 31, 1971",1971,sie tötete in ekstase,0.0,0.0,0.0,0.0,12
0,"Dec 31, 1972",1972,whoever slew auntie roo?,0.0,0.0,0.0,0.0,12


In [549]:
# filter out box office revenues and budgets of zero

def remove_less_than(df, df_col, criterion):
    the_filter = df[df_col] > criterion
    df = df[the_filter]
    return df

df_numbers = remove_less_than(df_numbers, 'budget', 0)
df_numbers = remove_less_than(df_numbers, 'total_bo', 0)

print df_numbers.shape
df_numbers.head(3)

KeyError: 'budget'

In [539]:
# replace spaces with hyphens
df_numbers['title'] = df_numbers['title'].str.replace(' ', '-')

print df_numbers.shape
df_numbers.head(3)

,date,year,title,budget,bo,bo_int,total_bo,month
0,"Dec 26, 1973",1973,the-exorcist,12000000.0,204868002.0,197867132.0,402735134.0,12
0,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,140000.0,26572439.0,0.0,26572439.0,10
0,"Jun 20, 1975",1975,jaws,12000000.0,260000000.0,210700000.0,470700000.0,6


In [540]:
# the-numbers inflation ratio doesn't make sense so just import cpi data from us gov http://www.bls.gov/cpi/
# assume box office revenues earned within each release year and that inflation in each year equals the cpi data that I imported

# read cpi csv
df_cpi =pd.read_csv('cpi.csv')
df_cpi['year'] = [int(float(x)) for x in df_cpi['year']] # cast to int

# merge cpi into numbers dataframe
df_numbers = pd.merge(df_numbers, df_cpi, on='year')

print df_numbers.shape
df_numbers.head(3)

,date,year,title,budget,bo,bo_int,total_bo,month,cpi,cpi_ratio
0,"Dec 26, 1973",1973,the-exorcist,12000000.0,204868002.0,197867132.0,402735134.0,12,44.425,0.187451
1,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,140000.0,26572439.0,0.0,26572439.0,10,49.317,0.208093
2,"Jun 20, 1975",1975,jaws,12000000.0,260000000.0,210700000.0,470700000.0,6,53.825,0.227114


In [541]:
# adjust for inflation

df_numbers['budget_i'] = df_numbers['budget']/df_numbers['cpi_ratio']
df_numbers['bo_i'] = df_numbers['bo']/df_numbers['cpi_ratio']
df_numbers['bo_int_i'] = df_numbers['bo_int']/df_numbers['cpi_ratio']
df_numbers['total_bo_i'] = df_numbers['total_bo']/df_numbers['cpi_ratio']

print df_numbers.shape
df_numbers.head(3)

,date,year,title,budget,bo,bo_int,total_bo,month,cpi,cpi_ratio,budget_i,bo_i,bo_int_i,total_bo_i
0,"Dec 26, 1973",1973,the-exorcist,12000000.0,204868002.0,197867132.0,402735134.0,12,44.425,0.187451,6.401666e+07,1.092914e+09,1.055566e+09,2.148480e+09
1,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,140000.0,26572439.0,0.0,26572439.0,10,49.317,0.208093,6.727761e+05,1.276950e+08,0.000000e+00,1.276950e+08
2,"Jun 20, 1975",1975,jaws,12000000.0,260000000.0,210700000.0,470700000.0,6,53.825,0.227114,5.283679e+07,1.144797e+09,9.277259e+08,2.072523e+09


In [542]:
# drop columns
df_numbers = df_numbers.drop(df_numbers.columns[[3, 4, 5, 6]], axis=1)

print df_numbers.shape
df_numbers.head(3)

,date,year,title,month,cpi,cpi_ratio,budget_i,bo_i,bo_int_i,total_bo_i
0,"Dec 26, 1973",1973,the-exorcist,12,44.425,0.187451,6.401666e+07,1.092914e+09,1.055566e+09,2.148480e+09
1,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,10,49.317,0.208093,6.727761e+05,1.276950e+08,0.000000e+00,1.276950e+08
2,"Jun 20, 1975",1975,jaws,6,53.825,0.227114,5.283679e+07,1.144797e+09,9.277259e+08,2.072523e+09


In [543]:
# calculate profit, profit margin, and % box office that's international
df_numbers['profit_i'] = df_numbers['total_bo_i']-df_numbers['budget_i']
df_numbers['margin'] = df_numbers['profit_i']/df_numbers['budget_i']
df_numbers['percent_int'] = df_numbers['bo_int_i']/df_numbers['total_bo_i']

print df_numbers.shape
df_numbers.head(3)

(275, 13)


,date,year,title,month,cpi,cpi_ratio,budget_i,bo_i,bo_int_i,total_bo_i,profit_i,margin,percent_int
0,"Dec 26, 1973",1973,the-exorcist,12,44.425,0.187451,6.401666e+07,1.092914e+09,1.055566e+09,2.148480e+09,2.084463e+09,32.561261,0.491308
1,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,10,49.317,0.208093,6.727761e+05,1.276950e+08,0.000000e+00,1.276950e+08,1.270222e+08,188.803136,0.000000
2,"Jun 20, 1975",1975,jaws,6,53.825,0.227114,5.283679e+07,1.144797e+09,9.277259e+08,2.072523e+09,2.019686e+09,38.225000,0.447631


In [544]:
# read in metacritic data
df_metacritic = pd.read_pickle('metacritic/consolidated.pickle')
df_metacritic.head(3)

,index,title,metascore,userscore
0,0,all-mistakes-buried,56,tbd
1,1,drunk-stoned-brilliant-dead-the-story-of-the-n...,74,tbd
2,2,fifty-shades-of-black,28,1.6


In [545]:
# drop index column
# drop columns
df_metacritic = df_metacritic.drop(df_metacritic.columns[[0]], axis=1)
print df_metacritic.shape
df_metacritic.head(3)

(7831, 3)


,title,metascore,userscore
0,all-mistakes-buried,56,tbd
1,drunk-stoned-brilliant-dead-the-story-of-the-n...,74,tbd
2,fifty-shades-of-black,28,1.6


In [546]:
# merge metacritic data with numbers data
df = pd.merge(df_numbers, df_metacritic, on='title')
df

,date,year,title,month,cpi,cpi_ratio,budget_i,bo_i,bo_int_i,total_bo_i,profit_i,margin,percent_int,metascore,userscore
0,"Dec 26, 1973",1973,the-exorcist,12,44.425,0.187451,6.401666e+07,1.092914e+09,1.055566e+09,2.148480e+09,2.084463e+09,32.561261,0.491308,82,8.8
1,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,10,49.317,0.208093,6.727761e+05,1.276950e+08,0.000000e+00,1.276950e+08,1.270222e+08,188.803136,0.000000,38,6.6
2,"Oct 17, 2003",2003,the-texas-chainsaw-massacre,10,184.000,0.776388,1.159215e+07,1.035437e+08,3.528606e+07,1.388297e+08,1.272376e+08,10.976187,0.254168,38,6.6
3,"Jun 20, 1975",1975,jaws,6,53.825,0.227114,5.283679e+07,1.144797e+09,9.277259e+08,2.072523e+09,2.019686e+09,38.225000,0.447631,79,8.7
4,"Nov 16, 1976",1976,carrie,11,56.933,0.240229,7.492860e+06,1.077230e+08,0.000000e+00,1.077230e+08,1.002301e+08,13.376752,0.000000,53,6.1
5,"Oct 18, 2013",2013,carrie,10,232.964,0.982991,3.051909e+07,3.587684e+07,4.795862e+07,8.383546e+07,5.331637e+07,1.746984,0.572056,53,6.1
6,"Jun 25, 1976",1976,the-omen,6,56.933,0.240229,1.165556e+07,2.021860e+08,0.000000e+00,2.021860e+08,1.905305e+08,16.346745,0.000000,43,6.4
7,"Jun 6, 2006",2006,the-omen,6,201.558,0.850474,2.939538e+07,6.420820e+07,7.606138e+07,1.402696e+08,1.108742e+08,3.771823,0.542251,43,6.4
8,"Oct 17, 1978",1978,halloween,10,65.242,0.275289,1.180580e+06,1.707300e+08,8.354871e+07,2.542787e+08,2.530981e+08,214.384615,0.328571,47,5.7
9,"Aug 31, 2007",2007,halloween,8,207.344,0.874888,1.714506e+07,6.660187e+07,2.199740e+07,8.859926e+07,7.145420e+07,4.167627,0.248280,47,5.7


In [547]:
# filter out movies that were not reviewed and any NaNs
# assume opinions held about movies now are same as they were when movies were released
def filter_out_not_equal(df, df_col, criterion):
    the_filter = df[df_col] != criterion
    df = df[the_filter]
    return df

df = filter_out_not_equal(df, 'userscore','tbd')
df.dropna(thresh=1)

df

,date,year,title,month,cpi,cpi_ratio,budget_i,bo_i,bo_int_i,total_bo_i,profit_i,margin,percent_int,metascore,userscore
0,"Dec 26, 1973",1973,the-exorcist,12,44.425,0.187451,6.401666e+07,1.092914e+09,1.055566e+09,2.148480e+09,2.084463e+09,32.561261,0.491308,82,8.8
1,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,10,49.317,0.208093,6.727761e+05,1.276950e+08,0.000000e+00,1.276950e+08,1.270222e+08,188.803136,0.000000,38,6.6
2,"Oct 17, 2003",2003,the-texas-chainsaw-massacre,10,184.000,0.776388,1.159215e+07,1.035437e+08,3.528606e+07,1.388297e+08,1.272376e+08,10.976187,0.254168,38,6.6
3,"Jun 20, 1975",1975,jaws,6,53.825,0.227114,5.283679e+07,1.144797e+09,9.277259e+08,2.072523e+09,2.019686e+09,38.225000,0.447631,79,8.7
4,"Nov 16, 1976",1976,carrie,11,56.933,0.240229,7.492860e+06,1.077230e+08,0.000000e+00,1.077230e+08,1.002301e+08,13.376752,0.000000,53,6.1
5,"Oct 18, 2013",2013,carrie,10,232.964,0.982991,3.051909e+07,3.587684e+07,4.795862e+07,8.383546e+07,5.331637e+07,1.746984,0.572056,53,6.1
6,"Jun 25, 1976",1976,the-omen,6,56.933,0.240229,1.165556e+07,2.021860e+08,0.000000e+00,2.021860e+08,1.905305e+08,16.346745,0.000000,43,6.4
7,"Jun 6, 2006",2006,the-omen,6,201.558,0.850474,2.939538e+07,6.420820e+07,7.606138e+07,1.402696e+08,1.108742e+08,3.771823,0.542251,43,6.4
8,"Oct 17, 1978",1978,halloween,10,65.242,0.275289,1.180580e+06,1.707300e+08,8.354871e+07,2.542787e+08,2.530981e+08,214.384615,0.328571,47,5.7
9,"Aug 31, 2007",2007,halloween,8,207.344,0.874888,1.714506e+07,6.660187e+07,2.199740e+07,8.859926e+07,7.145420e+07,4.167627,0.248280,47,5.7


In [548]:
# fix scores for texas chainsaw massacre 1974
df.iloc[1,-2] = 78
df.iloc[1,-1] = 7.9

# use rotten tomatoes rank for carrie
# http://www.rottentomatoes.com/m/1003625-carrie/
df.iloc[4,-2] = 93
df.iloc[4,-1] = 7.6

# use rotten tomatoes rank for the omen
# http://www.rottentomatoes.com/m/1015517-omen/
df.iloc[6,-2] = 86
df.iloc[6,-1] = 8.0

# use rotten tomatoes rank for dawn of the dead
# http://www.rottentomatoes.com/m/1005339-dawn_of_the_dead/
df.iloc[10,-2] = 92
df.iloc[10,-1] = 9.0

# use rotten tomatoes rank for friday-the-13th
# http://www.rottentomatoes.com/m/friday_the_13th_part_1/
df.iloc[13,-2] = 58
df.iloc[13,-1] = 6.1

# use rotten tomatoes rank for the fog user score
# http://www.rottentomatoes.com/m/1007617-fog/
df.iloc[15,-2] = 74
df.iloc[15,-1] = 5.8

# use rotten tomatoes rank for poltergeist
# http://www.rottentomatoes.com/m/1016513-poltergeist/
df.iloc[18,-2] = 88
df.iloc[18,-1] = 7.8




df

,date,year,title,month,cpi,cpi_ratio,budget_i,bo_i,bo_int_i,total_bo_i,profit_i,margin,percent_int,metascore,userscore
0,"Dec 26, 1973",1973,the-exorcist,12,44.425,0.187451,6.401666e+07,1.092914e+09,1.055566e+09,2.148480e+09,2.084463e+09,32.561261,0.491308,82,8.8
1,"Oct 18, 1974",1974,the-texas-chainsaw-massacre,10,49.317,0.208093,6.727761e+05,1.276950e+08,0.000000e+00,1.276950e+08,1.270222e+08,188.803136,0.000000,78,7.9
2,"Oct 17, 2003",2003,the-texas-chainsaw-massacre,10,184.000,0.776388,1.159215e+07,1.035437e+08,3.528606e+07,1.388297e+08,1.272376e+08,10.976187,0.254168,38,6.6
3,"Jun 20, 1975",1975,jaws,6,53.825,0.227114,5.283679e+07,1.144797e+09,9.277259e+08,2.072523e+09,2.019686e+09,38.225000,0.447631,79,8.7
4,"Nov 16, 1976",1976,carrie,11,56.933,0.240229,7.492860e+06,1.077230e+08,0.000000e+00,1.077230e+08,1.002301e+08,13.376752,0.000000,93,7.6
5,"Oct 18, 2013",2013,carrie,10,232.964,0.982991,3.051909e+07,3.587684e+07,4.795862e+07,8.383546e+07,5.331637e+07,1.746984,0.572056,53,6.1
6,"Jun 25, 1976",1976,the-omen,6,56.933,0.240229,1.165556e+07,2.021860e+08,0.000000e+00,2.021860e+08,1.905305e+08,16.346745,0.000000,86,8
7,"Jun 6, 2006",2006,the-omen,6,201.558,0.850474,2.939538e+07,6.420820e+07,7.606138e+07,1.402696e+08,1.108742e+08,3.771823,0.542251,43,6.4
8,"Oct 17, 1978",1978,halloween,10,65.242,0.275289,1.180580e+06,1.707300e+08,8.354871e+07,2.542787e+08,2.530981e+08,214.384615,0.328571,47,5.7
9,"Aug 31, 2007",2007,halloween,8,207.344,0.874888,1.714506e+07,6.660187e+07,2.199740e+07,8.859926e+07,7.145420e+07,4.167627,0.248280,47,5.7
